# Input Loading

In [2]:
import pandas as pd

print('loading train...', flush=True, end='')

# x를 loading해서 (batch_size, step, channel)
input_path = '../dataset/preprocess4/input2/'
x_train = np.load(input_path+'x_train.npz', allow_pickle=True)['arr_0']
x_test = np.load(input_path+'x_test.npz', allow_pickle=True)['arr_0']
x_val = np.load(input_path+'x_val.npz', allow_pickle=True)['arr_0']

gender_train = np.load(input_path+'gender_train.npz', allow_pickle=True)['arr_0']
gender_test = np.load(input_path+'gender_test.npz', allow_pickle=True)['arr_0']
gender_val = np.load(input_path+'gender_val.npz', allow_pickle=True)['arr_0']

age_train = np.load(input_path+'age_train.npz', allow_pickle=True)['arr_0']
age_test = np.load(input_path+'age_test.npz', allow_pickle=True)['arr_0']
age_val = np.load(input_path+'age_val.npz', allow_pickle=True)['arr_0']

y_train = np.load(input_path+'y_train.npz')['arr_0']
y_test = np.load(input_path+'y_test.npz')['arr_0']
y_val = np.load(input_path+'y_val.npz')['arr_0']
print('done', flush=True)

print('x_train shape:', x_train.shape)
print('x_test.shape:', x_test.shape)
print('x_val.shape:', x_val.shape)

# binary classification
y_train_bin = y_train >= 4
y_test_bin = y_test >= 4
y_val_bin = y_val >= 4

loading train...done
x_train shape: (74636, 5000, 2)
x_test.shape: (8795, 5000, 2)
x_val.shape: (7955, 5000, 2)


In [ ]:
x_train_ppg = x_train[:,:,0:1]
x_val_ppg = x_val[:,:,0:1]
x_test_ppg = x_test[:,:,0:1]

x_train_ecg = x_train[:,:,1:2]
x_val_ecg = x_val[:,:,1:2]
x_test_ecg = x_test[:,:,1:2]

# GPU settings

In [ ]:
from keras.models import Sequential
from keras.models import Model, load_model
from keras.optimizers import Adam as Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPool1D, BatchNormalization, Dropout, Activation
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D, LeakyReLU, UpSampling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import os, pickle
from keras import backend as K
from keras import regularizers
from keras.initializers import he_normal, GlorotNormal


import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
# folder
nfold = 1  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 100
rootdir = "ECG/pacu/regression/1D_CNN_3layers_train_w_samp3_lowess_filtered"

predirs = []
for root, dirs, files in os.walk(rootdir):  # 하위 대상들을 recursive 하게 긁어옴
    for filename in dirs:
        predirs.append(filename)

if not os.path.exists(rootdir):
    os.mkdir(rootdir)


# test_settings
test_settings_1, test_settings_2, test_settings_3, test_settings_4 = [], [], [], []


# hyperparamters
#num_nodes = [64, 64, 64] #, 64, 64, 64]
#kernel_size = 10
pool_size = 2

#dense_node = 32
#dropout_rate = 0.2
learning_rate = 0.002

# hyperparamters pool
num_opts = [32, 64, 128, 256] # num of filters(kernel)
stride_opts = [1,1,1,1,1,2,2,2,2,3]
kernel_opts = [3,3,3,3,3,5,5,5,5,7] # kernel size
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [16, 32, 64, 128]
globalpool_opts = ['max','ave']
BATCH_SIZE = [512, 1024]


print('start making test settings...', end='', flush=True)
# test settings
for num_l1 in num_opts:
    for num_l2 in num_opts:
        for num_l3 in num_opts:
            for num_l4 in num_opts:
                for kernel_l1 in kernel_opts:
                    for kernel_l2 in kernel_opts:
                        for kernel_l3 in kernel_opts:
                            for kernel_l4 in kernel_opts:
                                test_settings_1.append([num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4])

for dense1 in dense_opts:
    for dropout1 in dropout_opts:
        for dropout_fc in dropout_opts:
            for globalpool_opt in globalpool_opts:
                for batch_size in BATCH_SIZE:
                    for conv_double in [True, False]:
                        test_settings_2.append([dense1, dropout1, dropout_fc, globalpool_opt, batch_size, conv_double])                                   

for stride_l1 in stride_opts:
    for stride_l2 in stride_opts:
        for stride_l3 in stride_opts:
            for stride_l4 in stride_opts:
                for stride_l5 in stride_opts:
                    for num_l5 in num_opts:
                        for kernel_l5 in kernel_opts:
                            test_settings_3.append([stride_l1, stride_l2, stride_l3, stride_l4, stride_l5, num_l5, kernel_l5])
      
for dense2 in dense_opts:
    for dropout2 in dropout_opts:
        test_settings_4.append([dense2, dropout2])
    
print('done')

# RandomSearch

In [ ]:
from keras import metrics
# random search for hyperparameter
ntrial = 200
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_acc = []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    test_setting_3 = random.choice(test_settings_3)
    test_setting_4 = random.choice(test_settings_4)    
    
        
    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense1, dropout1, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2
    stride_l1, stride_l2, stride_l3, stride_l4, stride_l5, num_l5, kernel_l5 = test_setting_3
    dense2, dropout2 = test_setting_4
    
    
    # total conv layers of the model
    n_conv = random.choice([2,3])
    
    if n_conv==2:
        num_l3,kernel_l3,stride_l3 = 0,0,0
        num_l4,kernel_l4,stride_l4 = 0,0,0
        num_l5,kernel_l5,stride_l5 = 0,0,0
    
    if n_conv==3:
        num_l4,kernel_l4,stride_l4 = 0,0,0
        num_l5,kernel_l5,stride_l5 = 0,0,0
        
    if n_conv==4:
        num_l5,kernel_l5,stride_l5 = 0,0,0  
        
    
    n_dense = random.choice([0,1,2])
    
    if n_dense==0:
        dense1, dropout1 = 0,0
        dense2, dropout2 = 0,0
    
    if n_dense==1:
        dense2, dropout2 = 0,0
        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={},c1={},c2={},c3={},filt1={},filt2={},filt3={},str1={},str2={},str3={},conv_double={},globalpool={},dropout={},dnodes={},dropout={},dnodes={},dropout={}'.format(batch_size, num_l1, num_l2, num_l3,kernel_l1, kernel_l2, kernel_l3, stride_l1,stride_l2,stride_l3,conv_double, globalpool_opt, dropout_fc, dense1, dropout1, dense2, dropout2)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/weights.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    with strategy.scope():
        # build a model
        inp_fnn = Input(shape=(agender_train.shape[1],))
        inp_cnn = Input(shape=(x_train.shape[1], x_train.shape[2]))
    
        out_fnn = inp_fnn
        out_fnn = Activation('sigmoid') (out_fnn)
        
        out_cnn = inp_cnn
        act='relu'
        
        
        # c1 layer
        if conv_double:
            out_cnn = Conv1D(filters=num_l1, kernel_size=kernel_l1, strides=stride_l1, padding='same') (out_cnn)
        out_cnn = Conv1D(filters=num_l1, kernel_size=kernel_l1, strides=stride_l1, padding='same') (out_cnn)
        out_cnn = Activation('relu') (out_cnn)
        out_cnn = BatchNormalization() (out_cnn)
        out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)


        # c2 layer
        if conv_double:
            out_cnn = Conv1D(filters=num_l2, kernel_size=kernel_l2, strides=stride_l2, padding='same') (out_cnn)
        out_cnn = Conv1D(filters=num_l2, kernel_size=kernel_l2, strides=stride_l2, padding='same') (out_cnn)
        out_cnn = Activation('relu') (out_cnn)
        out_cnn = BatchNormalization() (out_cnn)
        out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
        
        
        # c3 layer
        if n_conv>2:
            if conv_double:
                out_cnn = Conv1D(filters=num_l3, kernel_size=kernel_l3, strides=stride_l3, padding='same') (out_cnn)
            out_cnn = Conv1D(filters=num_l3, kernel_size=kernel_l3, strides=stride_l3, padding='same') (out_cnn)
            out_cnn = Activation('relu') (out_cnn)
            out_cnn = BatchNormalization() (out_cnn)
            out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
        
        
        # c4 layer
        if n_conv>3:
            if num_l3 == 512:
                model.add(Conv1D(filters=128,kernel_size=1,padding='same'))
            if conv_double:
                out_cnn = Conv1D(filters=num_l4, kernel_size=kernel_l4, strides=stride_l4, padding='same') (out_cnn)
            out_cnn = Conv1D(filters=num_l4, kernel_size=kernel_l4, strides=stride_l4, padding='same') (out_cnn)
            out_cnn = Activation('relu') (out_cnn)
            out_cnn = BatchNormalization() (out_cnn)
            out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
            
                    


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            out_cnn = GlobalMaxPool1D() (out_cnn)
        elif globalpool_opt == 'ave':
            out_cnn = GlobalAveragePooling1D() (out_cnn)    
        out_cnn = Dropout(dropout_fc) (out_cnn)
        
        
        out = concatenate([out_fnn, out_cnn])
            
        if n_dense>=1:
            out = Dense(dense1, activation='tanh') (out)
            out = Dropout(dropout1) (out)
            
        if n_dense==2:
            out = Dense(dense2, activation='tanh') (out)
            out = Dropout(dropout2) (out)
            
        out = Dense(1, activation='sigmoid') (out)
        
        
        model = Model(inputs=[inp_cnn, inp_fnn], outputs=[out])


        # model 학습 설정
        try:
            model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_trains, y_train_bin, sample_weight=train_w_samp, validation_data=(x_vals, y_val_bin, val_w_samp), epochs=50, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])
        except Exception as e:
            print(e)
            os.rmdir(odir)
            #os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            itrial -= 1
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue
            
            
    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_tests).flatten()

    
    # acc 계산
    acc = metrics.Accuracy()
    acc.update_state(y_pred>=0.5, y_test_bin, sample_weight=test_w_samp)
    acc_val = acc.result().numpy()
    test_acc.append(acc_val)
    
    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred, sample_weight=test_w_samp)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)

    # auprc 
    precision, recall, _ = precision_recall_curve(y_test_bin, y_pred, sample_weight=test_w_samp)
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)

    
    # rename
    os.rename(odir, rootdir+'/roc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))    
    
    
    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    
    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_roc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_roc(max_idx), random_settings(max_idx)))
